## Setup

In [2]:
source("/path/to/the/project/02_notebooks/00_setup.r")

In [3]:
p_load("DESeq2", "dplyr", install = FALSE)

In [4]:
metadata <- read.csv("2022_08_12_CDL_regression_sample_annotation.csv", row.names = 1)
data <- read.csv("03_outputs/02/count_matrix_annotated.csv", row.names = 1)
count_matrix <- data 

In [5]:
metadata <- metadata %>%
  tidyr::unite("Model_group", Model:Group, remove = FALSE)

In [6]:
metadata <- metadata %>%
  tidyr::unite("Model_regression", Model,Regression, remove = FALSE)

## Differential expression

### Merged dds - Cpos vs Cneg

In [8]:
dds_combined <- DESeqDataSetFromMatrix(countData = count_matrix, colData = metadata,
  design = ~Model_group)
dds_combined <- DESeq(dds_combined)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [7]:
res_TAA_pos_cneg <- lfcShrink(dds_combined, type="ashr", contrast = c("Model_group", "TAA_Cpos", "TAA_Cneg"))
res_TAA_cpos_cneg_sig <- subset(res_TAA_pos_cneg, padj < 0.01)

res_CCL4_cpos_cneg <- lfcShrink(dds_combined, type="ashr", contrast = c("Model_group", "CCL4_Cpos", "CCL4_Cneg"))
res_CCL4_cpos_cneg_sig <- subset(res_CCL4_cpos_cneg, padj < 0.01)

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041



In [8]:
TAA_cpos_cneg_sig_up <- subset(res_TAA_cpos_cneg_sig, log2FoldChange > 1.5)
TAA_cpos_cneg_sig_down <- subset(res_TAA_cpos_cneg_sig, log2FoldChange < -1.5)

CCL4_cpos_cneg_sig_up <- subset(res_CCL4_cpos_cneg_sig, log2FoldChange > 1.5)
CCL4_cpos_cneg_sig_down <- subset(res_CCL4_cpos_cneg_sig, log2FoldChange < -1.5)

### Merged dds - Reg vs Cpos

In [39]:
dds_combined_reg <- DESeqDataSetFromMatrix(countData = count_matrix, colData = metadata,
  design = ~Model_regression)
dds_combined_reg <- DESeq(dds_combined_reg)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 93 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [42]:
res_TAA_reg_pos <- lfcShrink(dds_combined_reg, type="ashr", contrast = c("Model_regression", "TAA_regression", "TAA_positive"))
res_TAA_reg_pos_sig <- subset(res_TAA_pos_reg, padj < 0.01)

res_CCL4_reg_pos <- lfcShrink(dds_combined_reg, type="ashr", contrast = c("Model_regression", "CCL4_regression", "CCL4_positive"))
res_CCL4_reg_pos_sig <- subset(res_CCL4_pos_reg, padj < 0.01)

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041



In [43]:
res_TAA_reg_pos_sig_up <- subset(res_TAA_reg_pos_sig, log2FoldChange > 1.5)
res_TAA_reg_pos_sig_down <- subset(res_TAA_reg_pos_sig, log2FoldChange < -1.5)

res_CCL4_reg_pos_sig_up <- subset(res_CCL4_reg_pos_sig, log2FoldChange > 1.5)
res_CCL4_reg_pos_sig_down <- subset(res_CCL4_reg_pos_sig, log2FoldChange < -1.5)

## Group-specific counts and metadata

In [9]:
ccl4_samples <- metadata[c(1:24),]

In [10]:
taa_samples <- metadata[c(25:48),]

In [11]:
ccl4_matrix <- count_matrix[, !colnames(count_matrix) %in% taa_samples$`Sample_id`]

In [12]:
taa_matrix <- count_matrix[, !colnames(count_matrix) %in% ccl4_samples$`Sample_id`]

## Group-specific dds

In [13]:
ccl4_samples <- ccl4_samples
taa_samples <- taa_samples

ccl4_matrix <- ccl4_matrix
taa_matrix <- taa_matrix

In [14]:
dds_ccl4 <- DESeqDataSetFromMatrix(countData = ccl4_matrix, colData = ccl4_samples,
  design = ~Group)
dds_ccl4 <- DESeq(dds_ccl4)

dds_taa <- DESeqDataSetFromMatrix(countData = taa_matrix, colData = taa_samples,
  design = ~Group)
dds_taa <- DESeq(dds_taa)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## CCL4 vs TAA

In [11]:
res_ccl4_taa_cir <- lfcShrink(dds_combined, type="ashr", contrast = c("Model_group", "CCL4_Cpos", "TAA_Cpos")) %>% as.data.frame() %>% arrange(padj)
res_ccl4_taa_cir <- res_ccl4_taa_cir %>% filter(padj < 0.05)

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041



In [12]:
res_ccl4_r2_r1 <- lfcShrink(dds_combined, type="ashr", contrast = c("Model_group", "CCL4_R2", "CCL4_R1")) %>% as.data.frame() %>% filter(padj < 0.05) %>% arrange(padj)
res_taa_r2_r1 <- lfcShrink(dds_combined, type="ashr", contrast = c("Model_group", "TAA_R2", "TAA_R1")) %>% as.data.frame() %>% filter(padj < 0.05) %>% arrange(padj)

res_ccl4_taa_r1_r1 <- lfcShrink(dds_combined, type="ashr", contrast = c("Model_group", "CCL4_R1", "TAA_R1")) %>% as.data.frame() %>% filter(padj < 0.05) %>% arrange(padj)
res_ccl4_taa_r2_r2 <- lfcShrink(dds_combined, type="ashr", contrast = c("Model_group", "CCL4_R2", "TAA_R2")) %>% as.data.frame() %>% filter(padj < 0.05) %>% arrange(padj)

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041



## Outputs

In [13]:
write.csv(res_ccl4_taa_cir, "03_outputs/03/res_ccl4_taa_cir.csv")
write.csv(res_ccl4_r2_r1, "03_outputs/03/res_ccl4_r2_r1.csv")
write.csv(res_taa_r2_r1, "03_outputs/03/res_taa_r2_r1.csv")
write.csv(res_ccl4_taa_r1_r1, "03_outputs/03/res_ccl4_taa_r1_r1.csv")
write.csv(res_ccl4_taa_r2_r2, "03_outputs/03/res_ccl4_taa_r2_r2.csv")

In [15]:
saveRDS(dds_combined, file = "03_outputs/03/dds_object_Model_group_wald.rds")
saveRDS(dds_ccl4, file = "03_outputs/03/dds_object_ccl4_group_wald.rds")
saveRDS(dds_taa, file = "03_outputs/03/dds_object_taa_group_wald.rds")

write.csv(as.data.frame(res_TAA_cpos_cneg_sig), 
          file="03_outputs/03/de_taa_cpos_cneg_sig_wald.csv")
write.csv(as.data.frame(res_CCL4_cpos_cneg_sig), 
          file="03_outputs/03/de_ccl4_cpos_cneg_sig_wald.csv")

write.csv(as.data.frame(TAA_cpos_cneg_sig_up), 
          file="03_outputs/03/de_taa_cpos_cneg_sig_up_wald.csv")
write.csv(as.data.frame(TAA_cpos_cneg_sig_down), 
          file="03_outputs/03/de_taa_cpos_cneg_sig_down_wald.csv")
write.csv(as.data.frame(CCL4_cpos_cneg_sig_up), 
          file="03_outputs/03/de_ccl4_cpos_cneg_sig_up_wald.csv")
write.csv(as.data.frame(CCL4_cpos_cneg_sig_down), 
          file="03_outputs/03/de_ccl4_cpos_cneg_sig_down_wald.csv")

In [ ]:
saveRDS(dds_ccl4_, file = "03_outputs/03/dds_object_ccl4_group_wald.rds")
saveRDS(dds_taa, file = "03_outputs/03/dds_object_taa_group_wald.rds")

In [50]:
write.csv(as.data.frame(res_TAA_reg_pos_sig), 
          file="03_outputs/03/de_taa_reg_pos_sig_wald.csv")
write.csv(as.data.frame(res_CCL4_reg_pos_sig), 
          file="03_outputs/03/de_ccl4_reg_pos_sig_wald.csv")

write.csv(as.data.frame(res_TAA_reg_pos_sig_up), 
          file="03_outputs/03/de_taa_reg_pos_sig_up_wald.csv")
write.csv(as.data.frame(res_TAA_reg_pos_sig_down), 
          file="03_outputs/03/de_taa_reg_pos_sig_down_wald.csv")
write.csv(as.data.frame(res_CCL4_reg_pos_sig_up), 
          file="03_outputs/03/de_ccl4_reg_pos_sig_up_wald.csv")
write.csv(as.data.frame(res_CCL4_reg_pos_sig_down), 
          file="03_outputs/03/de_ccl4_reg_pos_sig_down_wald.csv")

In [16]:
write.csv(as.data.frame(ccl4_samples), 
          file="03_outputs/03/ccl4_samples_metadata_wald.csv")
write.csv(as.data.frame(taa_samples), 
          file="03_outputs/03/taa_samples_metadata_wald.csv")

write.csv(as.data.frame(ccl4_matrix), 
          file="03_outputs/03/ccl4_annotated_genes_expression_matrix_wald.csv")
write.csv(as.data.frame(taa_matrix), 
          file="03_outputs/03/taa_annotated_genes_expression_matrix_wald.csv")